In [42]:
import numpy as np
import pandas as pd

In [45]:
data=pd.read_csv("./diabetes2.csv")
X = data.drop('Outcome', axis=1).values
y = data['Outcome'].values * 2 - 1 

X = (X - X.mean(axis=0)) / X.std(axis=0)  # Normalize features
X = np.hstack((np.ones((X.shape[0], 1)), X))  # Add bias term


In [46]:
def p(a, x):
    return 1 / (1 + np.exp(-np.dot(a, x)))

In [47]:
def loss(X, y, w):
    return -np.sum(
        np.where(y == 1, np.log(p(w, X.T)), np.log(1 - p(w, X.T)))
    )

In [48]:
def gradient(X, y, w):
    pred = p(w, X.T)
    return -X.T.dot(y - pred)

In [49]:
# Gradient Descent with Inexact Line Search
def gradient_descent(X, y, max_iter=1000, tol=1e-6):
    w = np.zeros(X.shape[1])
    
    for i in range(max_iter):
        grad = gradient(X, y, w)
        if np.linalg.norm(grad) < tol:
            break
        
        # Backtracking line search
        alpha = 1.0
        beta = 0.5
        c = 0.1
        while loss(X, y, w - alpha * grad) > loss(X, y, w) - c * alpha * np.dot(grad, grad):
            alpha *= beta
        
        w -= alpha * grad
    
    return w


In [50]:
# Mirror Descent
def mirror_descent(X, y, Q, max_iter=10000, tol=1e-6):
    w = np.zeros(X.shape[1])
    theta = np.zeros(X.shape[1])
    for i in range(max_iter):
        grad = gradient(X, y, w)
        if np.linalg.norm(grad) < tol:
            break
        
        theta -= Q.dot(grad)
        w = np.linalg.solve(Q, theta)
    return w


In [51]:
# Construct Q matrix
n_features = X.shape[1]
Q = np.diag(np.random.uniform(0, 10, n_features))
Q[np.triu_indices(n_features, 1)] = np.random.uniform(0, 1, (n_features * (n_features - 1)) // 2)
Q = (Q + Q.T) / 2  # Make symmetric

In [52]:
w_gd=gradient_descent(X,y)
print("Gradient Descent weights:", w_gd)

C:\Users\HP\AppData\Local\Temp\ipykernel_30532\3531335829.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-np.dot(a, x)))
C:\Users\HP\AppData\Local\Temp\ipykernel_30532\2226549035.py:3: RuntimeWarning: divide by zero encountered in log
  np.where(y == 1, np.log(p(w, X.T)), np.log(1 - p(w, X.T)))


Gradient Descent weights: [-1.203125    0.31729711  0.66717514  0.0930427   0.10688988  0.18667343
  0.41853062  0.2485835   0.34083053]


In [53]:
w_md=mirror_descent(X,y,Q)
print("Mirror Descent weights:", w_md)

C:\Users\HP\AppData\Local\Temp\ipykernel_30532\3531335829.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-np.dot(a, x)))


Mirror Descent weights: [-3507043.36389206   774007.16761818  1794909.25409977   -30619.7002288
    80522.6021505    242766.20594596  1233096.76351494   522962.13473405
   799128.75262708]


In [41]:
# Evaluate models
def accuracy(X, y, w):
    return np.mean((sigmoid(X.dot(w)) >= 0.5) == y)

print("Gradient Descent accuracy:", accuracy(X, y, w_gd))
print("Mirror Descent accuracy:", accuracy(X, y, w_md))

Gradient Descent accuracy: 0.6940104166666666
Mirror Descent accuracy: 0.6953125


C:\Users\HP\AppData\Local\Temp\ipykernel_30532\3196251242.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [55]:
import numpy as np
import pandas as pd

# Load the data
data = pd.read_csv("./diabetes2.csv")
X = data.drop('Outcome', axis=1).values
y = data['Outcome'].values * 2 - 1  # Convert to {-1, 1}
X = (X - X.mean(axis=0)) / X.std(axis=0)  # Normalize features
X = np.hstack((np.ones((X.shape[0], 1)), X))  # Add bias term

def p(a, x):
    return 1 / (1 + np.exp(-np.dot(a, x)))

def loss(X, y, w):
    return -np.sum(
        np.where(y == 1, np.log(p(w, X.T)), np.log(1 - p(w, X.T)))
    )

def gradient(X, y, w):
    pred = p(w, X.T)
    return -X.T.dot(y - pred)

# Gradient Descent with Custom Inexact Line Search
def gradient_descent(X, y, max_iter=1000, tol=1e-6):
    w = np.zeros(X.shape[1])
    
    for i in range(max_iter):
        grad = gradient(X, y, w)
        if np.linalg.norm(grad) < tol:
            break
        
        # Backtracking line search
        alpha = 1.0
        beta = 0.5
        c = 0.1
        while loss(X, y, w - alpha * grad) > loss(X, y, w) - c * alpha * np.dot(grad, grad):
            alpha *= beta
        
        w -= alpha * grad
    
    return w

# Mirror Descent
def mirror_descent(X, y, Q, max_iter=1000, tol=1e-6):
    w = np.zeros(X.shape[1])
    theta = np.zeros(X.shape[1])
    for i in range(max_iter):
        grad = gradient(X, y, w)
        if np.linalg.norm(grad) < tol:
            break
        
        theta -= Q.dot(grad)
        w = np.linalg.solve(Q, theta)
    return w

# Construct Q matrix
n_features = X.shape[1]
Q = np.diag(np.random.uniform(0, 10, n_features))
Q[np.triu_indices(n_features, 1)] = np.random.uniform(0, 1, (n_features * (n_features - 1)) // 2)
Q = (Q + Q.T) / 2  # Make symmetric

# Run both methods
w_gd = gradient_descent(X, y)
w_md = mirror_descent(X, y, Q)

print("Gradient Descent weights:", w_gd)
print("Mirror Descent weights:", w_md)

# Evaluate models
def accuracy(X, y, w):
    return np.mean((p(w, X.T) >= 0.5) == (y == 1))

print("Gradient Descent accuracy:", accuracy(X, y, w_gd))
print("Mirror Descent accuracy:", accuracy(X, y, w_md))

C:\Users\HP\AppData\Local\Temp\ipykernel_30532\740857915.py:12: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-np.dot(a, x)))
C:\Users\HP\AppData\Local\Temp\ipykernel_30532\740857915.py:16: RuntimeWarning: divide by zero encountered in log
  np.where(y == 1, np.log(p(w, X.T)), np.log(1 - p(w, X.T)))


Gradient Descent weights: [-1.203125    0.31729711  0.66717514  0.0930427   0.10688988  0.18667343
  0.41853062  0.2485835   0.34083053]
Mirror Descent weights: [-350968.96477502   77464.47385853  179615.46173951   -3033.16131793
    8077.21001815   24295.05185938  123392.10759004   52361.48443743
   79977.24804499]
Gradient Descent accuracy: 0.7434895833333334
Mirror Descent accuracy: 0.7434895833333334
